In [1]:
import os
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from dotenv import load_dotenv
import tensorflow as tf
import tensorflow_hub as hub
import pydicom as dicom
from skimage.transform import resize
import cv2
import numpy as np

In [2]:
UPLOAD_FOLDER = 'static/uploads/'
ENDPOINT = os.environ['ENDPOINT']
prediction_key = os.environ['PREDICTION_KEY']
project_id = os.environ['PROJECT_ID']

In [3]:
prediction_credentials = ApiKeyCredentials(
    in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [4]:
publish_iteration_name = "Iteration2"
PNG = False

In [5]:
def predict(image_path):
    model_h5 = tf.keras.models.load_model(
    './model_dicom_cancer.h5', custom_objects={'KerasLayer': hub.KerasLayer})

    pred90 = model_h5.predict(image_path.reshape(1,256, 256, 1))
    pred90 = pred90[0][1]*100
    if int(pred90) < 90:
        val='Normal'
    else:
        val='Cancer'
    return val

In [25]:
file_url="E:\\GSoC\\ct_imaging\\cancer_app\\static\\uploads\\ID_0007_AGE_0061_CONTRAST_1_CT.dcm"
ds = dicom.dcmread(file_url)
test90 = ds.pixel_array
IMG_PX_SIZE = 256
resized90 = resize(test90, (IMG_PX_SIZE, IMG_PX_SIZE, 1), anti_aliasing=True)

test91 = ds.pixel_array.astype(float)
threshold = 500
test91 = (np.maximum(test91, 0) / (np.amax(test91) + threshold)) * 255.0

In [13]:
if PNG == False:
    filename = file_url.replace('.dcm', '.jpg')
else:
    filename = file_url.replace('.dcm', '.png')
cv2.imwrite((UPLOAD_FOLDER, file_url), test91)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - Can't convert object to 'str' for 'filename'
>  - Can't convert object to 'str' for 'filename'


In [26]:
# with open(os.path.join(app.config['UPLOAD_FOLDER'], filename), 'rb') as image_contents:
ct_image=''
filename = "E:\\GSoC\\ct_imaging\\cancer_app\\static\\uploads\\ID_0007_AGE_0061_CONTRAST_1_CT.jpg"
with open(filename, 'rb') as image_contents:
    results=predictor.classify_image(project_id, publish_iteration_name, image_contents.read())
    for prediction in results.predictions:
        if prediction.probability * 100 > 99:
            if prediction.tag_name == "chest":
                # print(prediction.probability * 100, " ", prediction.tag_name)
                ct_image=predict(resized90)
                ct_image=''.join(ct_image)
            else:
                ct_image='Non-chest DCM'
        else:
            ct_image='not Medical DCM'

print(ct_image)

1/1 [==============================] - 0s 90ms/step
not Medical DCM
